In [17]:
import pandas as pd

In [62]:
def import_and_transform_air_quality_data():
    streets = ['Zch_Rosengartenstrasse', 'Zch_Stampfenbachstrasse', 'Zch_Schimmelstrasse']
    street_dataframes = []

    for street in streets:
        yearly_data = []

        for year in range(2012, 2023):
            file_path = f'airquality/daily/ugz_ogd_air_d1_{year}.csv'
            data = pd.read_csv(file_path)

            # Filter for a specific street
            street_data = data[data['Standort'] == street]

            # Pivot the data
            street_wide = street_data.pivot(index='Datum', columns='Parameter', values='Wert')

            # Add the year and street columns
            street_wide['Year'] = year
            street_wide['Street'] = street

            yearly_data.append(street_wide)

        # Concatenate all years' data for the street
        combined_street_data = pd.concat(yearly_data)
        street_dataframes.append(combined_street_data)

    # Concatenate all streets' data into one DataFrame
    combined_data = pd.concat(street_dataframes)

    return combined_data

In [67]:
airquality = import_and_transform_air_quality_data()

In [23]:
def import_and_transform_traffic_data():
    streets = ['Stampfenbachstrasse (Wasserwerkstrasse)', 'Schimmelstrasse (Werdstrasse)', 'Rosengartenstrasse']
    street_dataframes = []

    for street in streets:
        yearly_data = []

        for year in range(2012, 2023):
            file_path = f'traffic/sid_dav_verkehrszaehlung_miv_od2031_{year}.csv'
            traffic = pd.read_csv(file_path)

            # Drop unnecessary columns and filter for a specific street
            traffic_small = traffic.drop(columns=['MSID', 'MSName', 'Hoehe', 'EKoord', 'NKoord', 'AnzDetektoren', 'D1ID', 'D2ID', 'D3ID','D4ID', 'Knummer', 'ZSID'])
            traffic_rg = traffic_small[traffic_small['ZSName'] == street]
            traffic_rg['MessungDatZeit'] = pd.to_datetime(traffic_rg['MessungDatZeit'])

            # Aggregate data
            daily_traffic = traffic_rg.groupby(traffic_rg['MessungDatZeit'].dt.date)['AnzFahrzeuge'].sum().reset_index(name='TotalDailyTraffic')
            daily_traffic['Year'] = year
            daily_traffic['Street'] = street

            yearly_data.append(daily_traffic)

        # Concatenate all years' data for the street
        combined_street_data = pd.concat(yearly_data)
        street_dataframes.append(combined_street_data)

    # Concatenate all streets' data into one DataFrame
    combined_data = pd.concat(street_dataframes)

    return combined_data


In [ ]:
traffic = import_and_transform_traffic_data()

In [ ]:
airquality = pd.read_csv('airquality/airquality.csv')
airquality['Datum'] = pd.to_datetime(airquality['Datum']).dt.date
#airquality.to_csv('airquality/airquality_dt.csv', index=False)


In [19]:
traffic = pd.read_csv('traffic/traffic_data.csv') 
traffic.replace(['Stampfenbachstrasse (Wasserwerkstrasse)','Schimmelstrasse (Werdstrasse)', 'Rosengartenstrasse'],['Zch_Stampfenbachstrasse', 'Zch_Schimmelstrasse', 'Zch_Rosengartenstrasse'], inplace=True)
traffic.rename(columns={"MessungDatZeit": "Datum"}, inplace=True)
#traffic.to_csv('traffic/traffic_rtm.csv', index=False)


In [14]:
airquality_dt = pd.read_csv('airquality/airquality_dt.csv')

In [ ]:
merged_df = pd.merge(traffic_rtm, airquality_dt, how="left", on=['Datum', 'Street'])
merged_df.drop(columns=merged_df[['Year_x', 'Year_y']])